In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("decisio_tree").getOrCreate()

In [3]:
data=spark.read.csv("/FileStore/tables/College.csv",inferSchema=True,header=True)

In [4]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
summary| School|Private| Apps| Accept| Enroll| Top10perc| Top25perc| F_Undergrad| P_Undergrad| Outstate| Room_Board| Books| Personal| PhD| Terminal| S_F_Ratio| perc_alumni| Expend| Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
 count| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777|
 mean| null| null|3001.6383526383524|2018.8043758043757|779.972972972973| 27.55855855855856| 55.7966537966538|3699.907335907336|855.2985842985843| 10440.66924066924| 4357.526383526383| 549.3809523809524|1340.6422136422136| 72.66023166023166| 79.70270270270271|14.089703989703986|22.743886743886744|9660.17117117117| 65.46332046332046|
 stddev| null| null|3870.2014844352884| 2451.11397099263| 929.17619013287|17.640364385452134|19.804777595131373|4850.420530887386|1522.431887295513|4023.0164841119727|1096.6964155935289|165.10536013709253| 677.071453590578|16.328154687939314|14.722358527903374|3.9583491352055478| 12.39180148937615|5221.76843985609|17.177709897155403|
 min|Abilene Christian...| No| 81| 72| 35| 1| 9| 139| 1| 2340| 1780| 96| 250| 8| 24| 2.5| 0| 3186| 10|
 max|York College of P...| Yes| 48094| 26330| 6392| 96| 100| 31643| 21836| 21700| 8124| 2340| 6800| 103| 100| 39.8| 64| 56233| 118|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+

In [5]:
data.head(1)

Out[ 7 ]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [6]:
data.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

Out[ 11 ]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler=VectorAssembler(inputCols=['Accept',
                                     'Enroll',
                                     'Top10perc',
                                     'Top25perc',
                                     'F_Undergrad',
                                     'P_Undergrad',
                                     'Outstate',
                                     'Room_Board',
                                     'Books',
                                     'Personal',
                                     'PhD',
                                     'Terminal',
                                     'S_F_Ratio',
                                     'perc_alumni',
                                     'Expend',
                                     'Grad_Rate'],outputCol='features')

In [10]:
output=assembler.transform(data)

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer=StringIndexer(inputCol="Private",outputCol='PrivateIndex')

In [13]:
output_fixed=indexer.fit(output).transform(output)

In [14]:
output_fixed.head(1)

Out[ 18 ]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]), PrivateIndex=0.0)]

In [15]:
final_data=output_fixed.select('features','PrivateIndex')

In [16]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [18]:
from pyspark.ml import Pipeline

In [19]:
dtc=DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc=RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt=GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [20]:
dtc_model=dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model=gbt.fit(train_data)

In [21]:
dtc_preds=dtc_model.transform(test_data)
rfc_preds=rfc_model.transform(test_data)
gbt_preds=gbt_model.transform(test_data)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_binary_eval=BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [24]:
print("DTC")
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.9010543840177581

In [25]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9733629300776917

In [26]:
my_binary_eval2=BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol='prediction')

In [27]:
print("GBT")
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8992785793562708

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
acc_eval=MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [30]:
rfc_acc=acc_eval.evaluate(rfc_preds)

In [31]:
rfc_acc

Out[ 40 ]: 0.9417040358744395